In [1]:
import pandas as pd
import numpy as np
import sklearn
print("✅ Everything works!")
print(f"Pandas: {pd.__version__}")
print(f"Scikit-learn: {sklearn.__version__}")

✅ Everything works!
Pandas: 2.2.2
Scikit-learn: 1.6.1


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

print("✅ All imports successful!")


✅ All imports successful!


In [3]:
# Load the datasets
train_data = pd.read_csv('Heart_Attack_training_dataset.csv')
test_data = pd.read_csv('Hear_Attack_evaluation_dataset')

# Show first few rows
print("Training data loaded!")
print(f"Number of rows: {len(train_data)}")
print(f"Number of columns: {len(train_data.columns)}")
print("\nFirst 5 rows:")
print(train_data.head())

FileNotFoundError: [Errno 2] No such file or directory: 'Hear_Attack_evaluation_dataset'

In [4]:
# Load the datasets
train_data = pd.read_csv('heart_train.csv')
test_data = pd.read_csv('heart_test.csv')

# Show first few rows
print("Training data loaded!")
print(f"Number of rows: {len(train_data)}")
print(f"Number of columns: {len(train_data.columns)}")
print("\nFirst 5 rows:")
print(train_data.head())


FileNotFoundError: [Errno 2] No such file or directory: 'heart_train.csv'

In [5]:
# Load the datasets
train_data = pd.read_csv('heart_train.csv')
test_data = pd.read_csv('heart_test.csv')

# Show first few rows
print("Training data loaded!")
print(f"Number of rows: {len(train_data)}")
print(f"Number of columns: {len(train_data.columns)}")
print("\nFirst 5 rows:")
print(train_data.head())


Training data loaded!
Number of rows: 7963
Number of columns: 26

First 5 rows:
  patient_id  age     sex  chol       bp  hr  diabetes  family_history  \
0    BMW7812   67    Male   208   158/88  72         0               0   
1    CZE1114   21    Male   389   165/93  98         1               1   
2    BNI9906   21  Female   324   174/99  72         1               0   
3    JLN3497   84    Male   383  163/100  73         1               1   
4    GFO8847   66    Male   318    91/88  93         1               1   

   smoking  obesity  ...  sedentary_hr  income        bmi  triglycerides  \
0        1        0  ...      6.615001  261404  31.251233            286   
1        1        1  ...      4.963459  285768  27.194973            235   
2        0        0  ...      9.463426  235282  28.176571            587   
3        1        0  ...      7.648981  125640  36.464704            378   
4        1        1  ...      1.514821  160555  21.809144            231   

   phys_act_days  

In [6]:
# Check for missing values
print("Missing values in each column:")
print(train_data.isnull().sum())

print("\n" + "="*50)
print("Target variable distribution:")
print(train_data['heart_attack_risk'].value_counts())

print("\n" + "="*50)
print("Basic statistics:")
print(train_data.describe())

Missing values in each column:
patient_id           0
age                  0
sex                  0
chol                 0
bp                   0
hr                   0
diabetes             0
family_history       0
smoking              0
obesity              0
alcohol              0
exercise_hr_wk       0
diet                 0
prev_heart_prob      0
med_use              0
stress_lvl           0
sedentary_hr         0
income               0
bmi                  0
triglycerides        0
phys_act_days        0
sleep_hr             0
country              0
continent            0
hemisphere           0
heart_attack_risk    0
dtype: int64

Target variable distribution:
heart_attack_risk
0    5224
1    2739
Name: count, dtype: int64

Basic statistics:
               age         chol           hr     diabetes  family_history  \
count  7963.000000  7963.000000  7963.000000  7963.000000     7963.000000   
mean     53.685922   259.574658    75.102474     0.653020        0.489640   
std      21.2

In [7]:
# Split blood pressure into systolic and diastolic
print("Original BP column sample:")
print(train_data['bp'].head())

# Split for training data
train_data[['bp_systolic', 'bp_diastolic']] = train_data['bp'].str.split('/', expand=True)
train_data['bp_systolic'] = pd.to_numeric(train_data['bp_systolic'])
train_data['bp_diastolic'] = pd.to_numeric(train_data['bp_diastolic'])
train_data = train_data.drop('bp', axis=1)

# Split for test data
test_data[['bp_systolic', 'bp_diastolic']] = test_data['bp'].str.split('/', expand=True)
test_data['bp_systolic'] = pd.to_numeric(test_data['bp_systolic'])
test_data['bp_diastolic'] = pd.to_numeric(test_data['bp_diastolic'])
test_data = test_data.drop('bp', axis=1)

print("\n✅ Blood pressure split into systolic and diastolic!")
print("New columns created:")
print(train_data[['bp_systolic', 'bp_diastolic']].head())

Original BP column sample:
0     158/88
1     165/93
2     174/99
3    163/100
4      91/88
Name: bp, dtype: object

✅ Blood pressure split into systolic and diastolic!
New columns created:
   bp_systolic  bp_diastolic
0          158            88
1          165            93
2          174            99
3          163           100
4           91            88


In [8]:
# Convert categorical (text) columns to numbers
categorical_cols = ['sex', 'diet', 'country', 'continent', 'hemisphere']

print("Converting text columns to numbers...")
for col in categorical_cols:
    le = LabelEncoder()
    # Fit on training data
    train_data[col] = le.fit_transform(train_data[col].astype(str))
    # Transform test data with same encoding
    test_data[col] = le.transform(test_data[col].astype(str))
    print(f"✅ {col} converted")

print("\n✅ All categorical variables converted to numbers!")
print("\nSample of converted data:")
print(train_data[['sex', 'diet', 'country']].head())

Converting text columns to numbers...
✅ sex converted
✅ diet converted
✅ country converted
✅ continent converted
✅ hemisphere converted

✅ All categorical variables converted to numbers!

Sample of converted data:
   sex  diet  country
0    1     0        0
1    1     2        3
2    0     1        6
3    1     0        3
4    1     2       16


In [9]:
# Separate features (X) and target (y)
X = train_data.drop(['patient_id', 'heart_attack_risk'], axis=1)
y = train_data['heart_attack_risk']

print(f"✅ Features prepared!")
print(f"Number of features: {X.shape[1]}")
print(f"Number of patients: {X.shape[0]}")
print(f"\nTarget distribution:")
print(y.value_counts())

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\n✅ Data split complete!")
print(f"Training set: {len(X_train)} patients")
print(f"Validation set: {len(X_val)} patients")

✅ Features prepared!
Number of features: 25
Number of patients: 7963

Target distribution:
heart_attack_risk
0    5224
1    2739
Name: count, dtype: int64

✅ Data split complete!
Training set: 6370 patients
Validation set: 1593 patients


In [10]:
# Scale features to similar ranges
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

print("✅ Features scaled successfully!")
print(f"Shape of training data: {X_train_scaled.shape}")
print(f"Shape of validation data: {X_val_scaled.shape}")

✅ Features scaled successfully!
Shape of training data: (6370, 25)
Shape of validation data: (1593, 25)


In [11]:
# Create and train the model
print("🚀 Training the model... (this may take 30-60 seconds)")

model = RandomForestClassifier(
    n_estimators=200,           # Number of trees
    max_depth=15,               # How deep each tree goes
    min_samples_split=10,       # Minimum samples to split
    min_samples_leaf=4,         # Minimum samples in leaf
    class_weight='balanced',    # Handle imbalanced data
    random_state=42,
    n_jobs=-1                   # Use all CPU cores
)

model.fit(X_train_scaled, y_train)

print("✅ Model training complete!")

🚀 Training the model... (this may take 30-60 seconds)
✅ Model training complete!


In [12]:
# Make predictions on validation set
y_pred = model.predict(X_val_scaled)
y_pred_proba = model.predict_proba(X_val_scaled)[:, 1]

# Calculate all metrics
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_pred_proba)

# Display results (THIS IS IMPORTANT - YOU NEED SCREENSHOT OF THIS!)
print("="*50)
print("📊 MODEL PERFORMANCE METRICS")
print("="*50)
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}  ⭐ MOST IMPORTANT!")
print(f"F1-Score:  {f1:.4f}")
print(f"ROC-AUC:   {roc_auc:.4f}")
print("="*50)

📊 MODEL PERFORMANCE METRICS
Accuracy:  0.6309
Precision: 0.3413
Recall:    0.0785  ⭐ MOST IMPORTANT!
F1-Score:  0.1276
ROC-AUC:   0.4997


In [13]:
# IMPROVED MODEL with focus on RECALL
print("🚀 Training IMPROVED model focused on HIGH RECALL...")

# Use different approach - focus on catching positive cases
model_improved = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,              # No limit on depth
    min_samples_split=2,         # More aggressive splitting
    min_samples_leaf=1,
    class_weight={0: 1, 1: 10},  # 10x weight on positive class!
    random_state=42,
    n_jobs=-1
)

model_improved.fit(X_train_scaled, y_train)

# Make predictions with lower threshold (catches more positives)
y_pred_proba_improved = model_improved.predict_proba(X_val_scaled)[:, 1]

# Use 0.3 threshold instead of 0.5 to catch more positive cases
y_pred_improved = (y_pred_proba_improved >= 0.3).astype(int)

# Calculate metrics
accuracy_imp = accuracy_score(y_val, y_pred_improved)
precision_imp = precision_score(y_val, y_pred_improved)
recall_imp = recall_score(y_val, y_pred_improved)
f1_imp = f1_score(y_val, y_pred_improved)
roc_auc_imp = roc_auc_score(y_val, y_pred_proba_improved)

print("="*50)
print("📊 IMPROVED MODEL PERFORMANCE")
print("="*50)
print(f"Accuracy:  {accuracy_imp:.4f}")
print(f"Precision: {precision_imp:.4f}")
print(f"Recall:    {recall_imp:.4f}  ⭐ MOST IMPORTANT!")
print(f"F1-Score:  {f1_imp:.4f}")
print(f"ROC-AUC:   {roc_auc_imp:.4f}")
print("="*50)
print(f"\n🎯 Recall improved from 0.0785 to {recall_imp:.4f}!")

🚀 Training IMPROVED model focused on HIGH RECALL...
📊 IMPROVED MODEL PERFORMANCE
Accuracy:  0.4846
Precision: 0.3436
Recall:    0.5474  ⭐ MOST IMPORTANT!
F1-Score:  0.4222
ROC-AUC:   0.5061

🎯 Recall improved from 0.0785 to 0.5474!


In [14]:
# FINAL MODEL - Maximum Recall Strategy
print("🚀 Training FINAL model with MAXIMUM RECALL focus...")

from sklearn.ensemble import GradientBoostingClassifier

# Try Gradient Boosting - often better for imbalanced data
model_final = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=5,
    min_samples_split=2,
    min_samples_leaf=1,
    subsample=0.8,
    random_state=42
)

model_final.fit(X_train_scaled, y_train)

# Get probabilities
y_pred_proba_final = model_final.predict_proba(X_val_scaled)[:, 1]

# Try even LOWER threshold - 0.25 (very aggressive)
y_pred_final = (y_pred_proba_final >= 0.25).astype(int)

# Calculate metrics
accuracy_final = accuracy_score(y_val, y_pred_final)
precision_final = precision_score(y_val, y_pred_final)
recall_final = recall_score(y_val, y_pred_final)
f1_final = f1_score(y_val, y_pred_final)
roc_auc_final = roc_auc_score(y_val, y_pred_proba_final)

print("="*50)
print("📊 FINAL MODEL PERFORMANCE")
print("="*50)
print(f"Accuracy:  {accuracy_final:.4f}")
print(f"Precision: {precision_final:.4f}")
print(f"Recall:    {recall_final:.4f}  ⭐ MOST IMPORTANT!")
print(f"F1-Score:  {f1_final:.4f}")
print(f"ROC-AUC:   {roc_auc_final:.4f}")
print("="*50)
print(f"\n🎯 Recall improved to {recall_final:.4f}!")

🚀 Training FINAL model with MAXIMUM RECALL focus...
📊 FINAL MODEL PERFORMANCE
Accuracy:  0.4175
Precision: 0.3356
Recall:    0.7080  ⭐ MOST IMPORTANT!
F1-Score:  0.4554
ROC-AUC:   0.5045

🎯 Recall improved to 0.7080!


In [15]:
# ULTRA-AGGRESSIVE MODEL - Maximum Recall Push
print("🚀 Training ULTRA-AGGRESSIVE model for MAXIMUM RECALL...")

# Retrain with even more extreme settings
model_ultra = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=7,
    min_samples_split=2,
    min_samples_leaf=1,
    subsample=1.0,
    random_state=42
)

model_ultra.fit(X_train_scaled, y_train)

# Try multiple thresholds to find the best
thresholds = [0.15, 0.20, 0.25, 0.30]
best_recall = 0
best_threshold = 0
best_metrics = {}

y_pred_proba_ultra = model_ultra.predict_proba(X_val_scaled)[:, 1]

print("\nTesting different thresholds:")
print("="*60)

for threshold in thresholds:
    y_pred_test = (y_pred_proba_ultra >= threshold).astype(int)

    recall_test = recall_score(y_val, y_pred_test)
    precision_test = precision_score(y_val, y_pred_test)
    f1_test = f1_score(y_val, y_pred_test)

    print(f"Threshold {threshold}: Recall={recall_test:.4f}, Precision={precision_test:.4f}, F1={f1_test:.4f}")

    if recall_test > best_recall:
        best_recall = recall_test
        best_threshold = threshold
        best_metrics = {
            'recall': recall_test,
            'precision': precision_test,
            'f1': f1_test,
            'accuracy': accuracy_score(y_val, y_pred_test)
        }

print("="*60)
print(f"\n🏆 BEST THRESHOLD: {best_threshold}")
print(f"🎯 BEST RECALL: {best_recall:.4f}")

🚀 Training ULTRA-AGGRESSIVE model for MAXIMUM RECALL...

Testing different thresholds:
Threshold 0.15: Recall=0.9325, Precision=0.3436, F1=0.5022
Threshold 0.2: Recall=0.8339, Precision=0.3457, F1=0.4888
Threshold 0.25: Recall=0.6880, Precision=0.3520, F1=0.4657
Threshold 0.3: Recall=0.5511, Precision=0.3540, F1=0.4311

🏆 BEST THRESHOLD: 0.15
🎯 BEST RECALL: 0.9325


In [16]:
# Install imbalanced-learn library
!pip install imbalanced-learn

from imblearn.over_sampling import SMOTE

print("🚀 Using SMOTE to balance the dataset...")

# Apply SMOTE to create synthetic samples
smote = SMOTE(random_state=42, k_neighbors=3)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print(f"\nOriginal training set:")
print(f"  Class 0 (No risk): {sum(y_train == 0)}")
print(f"  Class 1 (At risk): {sum(y_train == 1)}")

print(f"\nAfter SMOTE:")
print(f"  Class 0 (No risk): {sum(y_train_smote == 0)}")
print(f"  Class 1 (At risk): {sum(y_train_smote == 1)}")

# Train model on balanced data
model_smote = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)

model_smote.fit(X_train_smote, y_train_smote)

# Test with multiple thresholds
y_pred_proba_smote = model_smote.predict_proba(X_val_scaled)[:, 1]

print("\n" + "="*60)
print("SMOTE MODEL - Testing thresholds:")
print("="*60)

best_recall_smote = 0
best_threshold_smote = 0

for threshold in [0.2, 0.25, 0.3, 0.35, 0.4]:
    y_pred_test = (y_pred_proba_smote >= threshold).astype(int)

    recall_test = recall_score(y_val, y_pred_test)
    precision_test = precision_score(y_val, y_pred_test)
    f1_test = f1_score(y_val, y_pred_test)

    print(f"Threshold {threshold}: Recall={recall_test:.4f}, Precision={precision_test:.4f}, F1={f1_test:.4f}")

    if recall_test > best_recall_smote:
        best_recall_smote = recall_test
        best_threshold_smote = threshold

print("="*60)
print(f"\n🏆 BEST SMOTE RECALL: {best_recall_smote:.4f} at threshold {best_threshold_smote}")

🚀 Using SMOTE to balance the dataset...

Original training set:
  Class 0 (No risk): 4179
  Class 1 (At risk): 2191

After SMOTE:
  Class 0 (No risk): 4179
  Class 1 (At risk): 4179

SMOTE MODEL - Testing thresholds:
Threshold 0.2: Recall=0.7847, Precision=0.3437, F1=0.4780
Threshold 0.25: Recall=0.6734, Precision=0.3455, F1=0.4567
Threshold 0.3: Recall=0.5511, Precision=0.3444, F1=0.4239
Threshold 0.35: Recall=0.4325, Precision=0.3440, F1=0.3832
Threshold 0.4: Recall=0.3266, Precision=0.3358, F1=0.3312

🏆 BEST SMOTE RECALL: 0.7847 at threshold 0.2


In [17]:
# Final predictions with SMOTE model
best_model = model_smote
best_thresh = best_threshold_smote

test_pred_proba_best = best_model.predict_proba(X_test_scaled)[:, 1]
test_predictions_best = (test_pred_proba_best >= best_thresh).astype(int)

# Show final metrics on validation set
y_pred_final = (model_smote.predict_proba(X_val_scaled)[:, 1] >= best_thresh).astype(int)

print("="*50)
print("📊 FINAL MODEL METRICS (SMOTE)")
print("="*50)
print(f"Accuracy:  {accuracy_score(y_val, y_pred_final):.4f}")
print(f"Precision: {precision_score(y_val, y_pred_final):.4f}")
print(f"Recall:    {recall_score(y_val, y_pred_final):.4f}  ⭐")
print(f"F1-Score:  {f1_score(y_val, y_pred_final):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_val, model_smote.predict_proba(X_val_scaled)[:, 1]):.4f}")
print("="*50)
print(f"Threshold used: {best_thresh}")


NameError: name 'X_test_scaled' is not defined

In [18]:
# Scale the test data using the same scaler fitted on the training data
X_test = test_data.drop('patient_id', axis=1) # Drop 'patient_id' from test data
X_test_scaled = scaler.transform(X_test)

print("✅ Test data scaled successfully!")
print(f"Shape of test data: {X_test_scaled.shape}")

✅ Test data scaled successfully!
Shape of test data: (800, 25)


In [19]:
# Final predictions with SMOTE model
best_model = model_smote
best_thresh = best_threshold_smote

test_pred_proba_best = best_model.predict_proba(X_test_scaled)[:, 1]
test_predictions_best = (test_pred_proba_best >= best_thresh).astype(int)

# Show final metrics on validation set
y_pred_final = (model_smote.predict_proba(X_val_scaled)[:, 1] >= best_thresh).astype(int)

print("="*50)
print("📊 FINAL MODEL METRICS (SMOTE)")
print("="*50)
print(f"Accuracy:  {accuracy_score(y_val, y_pred_final):.4f}")
print(f"Precision: {precision_score(y_val, y_pred_final):.4f}")
print(f"Recall:    {recall_score(y_val, y_pred_final):.4f}  ⭐")
print(f"F1-Score:  {f1_score(y_val, y_pred_final):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_val, model_smote.predict_proba(X_val_scaled)[:, 1]):.4f}")
print("="*50)
print(f"Threshold used: {best_thresh}")

📊 FINAL MODEL METRICS (SMOTE)
Accuracy:  0.4105
Precision: 0.3437
Recall:    0.7847  ⭐
F1-Score:  0.4780
ROC-AUC:   0.5087
Threshold used: 0.2


In [20]:
# Make FINAL predictions on test data using the best model
test_pred_proba_final = model_final.predict_proba(X_test_scaled)[:, 1]
test_predictions_final = (test_pred_proba_final >= 0.25).astype(int)

print(f"✅ FINAL predictions complete!")
print(f"\nPrediction distribution:")
print(f"No risk (0): {sum(test_predictions_final == 0)}")
print(f"At risk (1): {sum(test_predictions_final == 1)}")
print(f"Total: {len(test_predictions_final)}")

✅ FINAL predictions complete!

Prediction distribution:
No risk (0): 207
At risk (1): 593
Total: 800


In [21]:
# Create FINAL submission file
submission_final = pd.DataFrame({
    'patient_id': test_data['patient_id'],
    'heart_attack_risk': test_predictions_final
})

print("FINAL Submission preview:")
print(submission_final.head(15))
print("\nChecking format...")
print(f"Columns: {list(submission_final.columns)}")
print(f"Total rows: {len(submission_final)}")

# IMPORTANT: Replace TeamCode and TeamName with YOUR actual team info!
submission_final.to_csv('EN17_TeamMatrix_Task1_Predictions.csv', index=False)

print("\n✅ FINAL CSV file created successfully!")

FINAL Submission preview:
   patient_id  heart_attack_risk
0     VRK5064                  1
1     NEN2365                  1
2     KXT2493                  1
3     TKO0406                  1
4     GDP2405                  0
5     GRQ8132                  1
6     KUL5067                  1
7     OTZ1268                  0
8     EZN4283                  0
9     HVE8034                  1
10    GIN1693                  1
11    LEG9626                  0
12    DWZ6826                  1
13    PHK4364                  1
14    VAZ4776                  1

Checking format...
Columns: ['patient_id', 'heart_attack_risk']
Total rows: 800

✅ FINAL CSV file created successfully!
